In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

In [ ]:
def feature_summary(data):
    n_row=data.shape[0]
    features=pd.DataFrame()
    features_names=[]
    features_type = []
    features_counts=[]
    features_missing=[]
    names=data.columns.values
    for i in names:
        features_names.append(i)
        features_type.append(type(data.ix[1,i]))
        features_counts.append(data[i].value_counts().count())
        features_missing.append(data[data[i].isnull()].shape[0])
    features['name']=features_names
    features['type'] = features_type
    features['value counts']=features_counts
    features['missing']=features_missing
    features['percentage_missing']=features['missing']/n_row
    return (features)

In [ ]:
feature_summary(df_train)

In [ ]:
df_train_quant = df_train.select_dtypes(include=['int64', 'floating', 'datetime64'])

In [ ]:
def normalize(test_series):
    normalized = (test_series - test_series.mean())/test_series.std()
    return normalized
def denormalize(test_series, std_val, mean_val):
    denormalized = (test_series*std_val) + mean_val
    return denormalized

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
PCA = PCA(n_components = 1)
#handling missing values using mean values? we can explore more methods for this at a later
#stage
df_train_quant = df_train_quant.drop('timestamp', 1)
for i in range(1,len(df_train_quant.columns)):
       df_train_quant.iloc[:,i] = df_train_quant.iloc[:,i].fillna(df_train_quant.iloc[:,i].mean())
y = df_train_quant['price_doc']
df_train_quant = df_train_quant.drop('price_doc', 1)
df_quant = PCA.fit_transform(df_train_quant)
y_mean = y.mean()
y_std = y.std()
y_normalized = normalize(y)

In [ ]:
#some PCA analysis
pd.DataFrame(PCA.explained_variance_ratio_).head(100)

In [ ]:
# simple linear regression
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm 
X_train,X_test,y_train,y_test = train_test_split(df_quant,y_normalized,test_size=0.2)# simple linear regression
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm 
#X_train,X_test,y_train,y_test = train_test_split(df_quant,y,test_size=0.2)

In [ ]:
#X_train = pd.DataFrame(X_train)
#X_test = pd.DataFrame(X_test)

In [ ]:
#model = lm.LinearRegression()
#m = model.fit(X_train, y_train)

In [ ]:
import xgboost as xgb
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [ ]:
# let us try random forests here? 
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators = 500)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
predy = clf.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,predy)

In [ ]:
# Transforming the test data
df_test_quant = df_test.select_dtypes(include=['int64', 'floating', 'datetime64'])
df_test_quant = df_test_quant.drop('timestamp', 1)
for i in range(1,len(df_test_quant.columns)):
       df_test_quant.iloc[:,i] = df_test_quant.iloc[:,i].fillna(df_test_quant.iloc[:,i].mean())
test = PCA.fit_transform(df_test_quant)

In [ ]:
y_values = m.predict(test)
y_values_denormalized = denormalize(y_values, y_std, y_mean)

In [ ]:
result = pd.DataFrame(y_values_denormalized).abs()

In [ ]:
# fitting results into a csv file
result['id'] = df_test['id']
result['price_doc'] = result.iloc[:,0]
result = result.drop(0,1)
result['price_doc'] = result['price_doc'].round(2)

In [ ]:
result.to_csv("output_7.csv", index = False)